<a href="https://colab.research.google.com/github/Mago281/Home_Sales/blob/main/Home_Sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import os
# # Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# # For example:
# # spark_version = 'spark-3.4.0'
# spark_version = 'spark-3.4.0'
# os.environ['SPARK_VERSION']=spark_version

# # Install Spark and Java
# !apt-get update
# !apt-get install openjdk-11-jdk-headless -qq > /dev/null
# !wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
# !tar xf $SPARK_VERSION-bin-hadoop3.tgz
# !pip install -q findspark

# # Set Environment Variables
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
# os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# # Start a SparkSession
# import findspark
# findspark.init()

In [4]:
!pip install pyspark==3.4.0

!pip install findspark pyarrow fastparquet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317122 sha256=5eaa6bed92629246dd18ff8f1c32f375221214c1bb2e8f2268378209562a1618
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.3 MB/s eta 0:00:00


In [5]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [6]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [7]:
# define 'home_sales_df' (Class 2 Activity 02)
spark.sparkContext.addFile(url)
home_sales_df = spark.read.csv(SparkFiles.get('home_sales_revised.csv'), sep = ",", header = True, ignoreLeadingWhiteSpace = True)
home_sales_df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [8]:
# 2. Create a temporary view of the DataFrame (Class 2 Activity 08)
home_sales_df.createOrReplaceTempView('home_sales_data')


In [9]:
print(home_sales_df)

DataFrame[id: string, date: string, date_built: string, price: string, bedrooms: string, bathrooms: string, sqft_living: string, sqft_lot: string, floors: string, waterfront: string, view: string]


In [10]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
# Class 1 Activity 07

spark.sql('''
  SELECT
    YEAR(date) YEAR,
    ROUND(AVG(price),2) Average_Price
  FROM home_sales_data
  WHERE bedrooms = 4
  GROUP BY YEAR
  ORDER BY YEAR DESC
''').show()



+----+-------------+
|YEAR|Average_Price|
+----+-------------+
|2022|    296363.88|
|2021|    301819.44|
|2020|    298353.78|
|2019|     300263.7|
+----+-------------+



In [11]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

avg_price_3_bed_3_bath = spark.sql('SELECT date_built, ROUND(AVG(price),2) AS Average_Price FROM home_sales_data where bedrooms = 3 AND bathrooms = 3 group by date_built ORDER BY date_built desc')
avg_price_3_bed_3_bath.show()


+----------+-------------+
|date_built|Average_Price|
+----------+-------------+
|      2017|    292676.79|
|      2016|    290555.07|
|      2015|     288770.3|
|      2014|    290852.27|
|      2013|    295962.27|
|      2012|    293683.19|
|      2011|    291117.47|
|      2010|    292859.62|
+----------+-------------+



In [23]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

avg_price_3_bed_3_bath_2_floors = '''
    SELECT
      Year(date_built) as year_built,
      ROUND(AVG(price),2) as average_price
    FROM home_sales_data
    WHERE bedrooms = 3
      AND bathrooms = 3
      AND floors = 2
      AND sqft_living >= 2000
    GROUP BY year_built
    ORDER BY year_built DESC
'''

# Execute the query using spark.sql
avg_price_3_bed_3_bath_2_floors_result = spark.sql(avg_price_3_bed_3_bath_2_floors)

# Show the result
avg_price_3_bed_3_bath_2_floors_result.show()


+----------+-------------+
|year_built|average_price|
+----------+-------------+
|      2017|    280317.58|
|      2016|     293965.1|
|      2015|    297609.97|
|      2014|    298264.72|
|      2013|    303676.79|
|      2012|    307539.97|
|      2011|    276553.81|
|      2010|    285010.22|
+----------+-------------+



In [25]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.
# Class 3 Activity 02

start_time = time.time()

spark.sql('''
    SELECT view, ROUND(AVG(price),2) as Average_Price
    FROM home_sales_data
    GROUP BY view
    HAVING AVG(price) >= 350000
    ORDER BY view desc
''').show()

print("--- %s seconds ---" % (time.time() - start_time))



+----+-------------+
|view|Average_Price|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
|  80|    991767.38|
+----+-------------+
only showing top 20 rows

--- 0.8923213481903076 seconds ---


In [14]:
# 7. Cache the the temporary table home_sales (Class 3 Activity 06)
spark.sql('cache table home_sales_data')

DataFrame[]

In [15]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales_data')

True

In [16]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000.
# Determine the runtime and compare it to uncached runtime (refer to the text box below).
# Class 3 Activity 02

start_time = time.time()

cached_data = '''
    SELECT
      view,
      ROUND(AVG(price),2) as Average_Price
    FROM home_sales_data
    GROUP BY view
    HAVING AVG(price) >= 350000
    ORDER BY view desc
  '''
spark.sql(cached_data).show()

print("--- %s seconds ---" % (time.time() - start_time))



+----+-------------+
|view|Average_Price|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
|  80|    991767.38|
+----+-------------+
only showing top 20 rows

--- 0.6312069892883301 seconds ---


The runtime for the cached data above was quicker at 0.6312069892883301 seconds while the uncached runtime was 0.8923213481903076 seconds.

In [17]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data (Class 3 Activity 01, 04)
# home_sales_df.write.parquet('home_parquet', mode = 'overwrite')

# Write out your parquet data, partitioning on the "date_built" field
home_sales_df.write.partitionBy('date_built').mode('overwrite').parquet('home_parquet')


In [18]:
# 11. Read the parquet formatted data.
parquet_home_sales_df = spark.read.parquet('home_parquet')

In [19]:
# 12. Create a temporary table for the parquet data.
parquet_home_sales_df.createOrReplaceTempView('temp_parquet_home_sales')

In [27]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version (refer to the text box below).

start_time = time.time()

query_view_rating = '''
    SELECT
      view,
      ROUND(AVG(price),2) as Average_Price
    FROM temp_parquet_home_sales
    GROUP BY view
    HAVING AVG(price) >= 350000
    ORDER BY view desc
    '''
spark.sql(query_view_rating).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|Average_Price|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
|  80|    991767.38|
+----+-------------+
only showing top 20 rows

--- 1.1581673622131348 seconds ---


The runtime for the above analysis is 1.1581673622131348 seconds while the runtime for the cached version was faster at 0.6312069892883301 seconds.

In [ ]:
# 14. Uncache the home_sales temporary table.
spark.sql('uncache table home_sales_data')

DataFrame[]

In [ ]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales_data')


False